In [ ]:
#Initialization
%matplotlib inline

import sys, os
import numpy as np
import math
from scipy.integrate import quad,dblquad
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib import rc, rcParams
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import clear_output

plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = 'gainsboro'

# Make use of TeX\ufeff
rc('text',usetex=True)
# Change all fonts to 'Computer Modern'
rc('font',**{'size':14, 'family':'serif','serif':['Times New Roman']})
rc('xtick.major', size=5, pad=7)
rc('xtick', labelsize=15)
rc('ytick.major', size=5, pad=7)
rc('ytick', labelsize=15)

# this needs to point to the folder where darkelf.py is stored
work_dir = os.getcwd()
sys.path.append(work_dir+"/..")
plotdir=work_dir+"/plots/"

from darkelf import darkelf, targets

#Velocities [km/s]
v_0   = 230
v_e   = 240
v_esc = 600
c     = 299792.458
v_max = (v_esc + v_e)/c

#Floating point error handling
eVtoInvS  = 1.5192674e15
yeartosec = 86400 * 365
eVtoInvYr = eVtoInvS * yeartosec

#Truncated Maxwell-Boltzmann Distribution
N_0 = np.pi**(3/2)*v_0**2*(v_0*math.erf(v_esc/v_0) - \
      2*v_esc/np.sqrt(np.pi)*np.exp(-v_esc**2/v_0**2))
a   = (np.pi**(3/2)*v_0**3)/(4*v_e*N_0)

def p1(v_z):
        return (a*(math.erf((v_e-v_z)/v_0) + math.erf((v_e+v_z)/v_0) - \
                (np.pi*v_0**2/N_0)*np.exp(-v_esc**2/v_0**2)))*(v_z < v_esc - v_e) + \
                (a*(math.erf(v_esc/v_0) + math.erf((v_e-v_z)/v_0)) - \
                (np.pi*v_0**2)/(2*N_0)*(v_e + v_esc - v_z)/v_e*np.exp(-v_esc**2/v_0**2)) * \
                (v_esc - v_e < v_z < v_esc + v_e)


#Data import
Si = darkelf(target='Si',filename='Si_mermin.dat',phonon_filename="Si_epsphonon_data6K.dat")
Ge = darkelf(target='Ge',filename='Ge_mermin.dat',phonon_filename="Ge_epsphonon_data2K.dat")
Al = darkelf(target='Al',filename='Al_mermin.dat')
SiO2 = darkelf(target='SiO2',filename='SiO2_mermin.dat',phonon_filename='SiO2_epsphonon_e.dat')

In [ ]:
#Figure 1 - ELF for Si and Ge
#(Routine made by Brian Campbell-Deem, Simon Knapen, Jonathan Kozaczuk, Tongyan Lin and Ethan Villarama)

kvals = np.linspace(10,1e4,400)*1e-3
ovals = np.linspace(0.01,80,160)
Nomega = len(ovals)
Nk = len(kvals)

z1_grid = np.zeros((Nomega,Nk))
z2_grid = np.zeros((Nomega,Nk))
ELF_Lindhard = np.zeros((Nomega,Nk))
ELF_mermin = np.zeros((Nomega,Nk))
ELF_gpaw = np.zeros((Nomega,Nk))

for j in range(Nk):
    for i in range(Nomega):
        ELF_gpaw[i][j] = Si.elf(ovals[i],kvals[j]*1e3)
        ELF_mermin[i][j] = Si.elf(ovals[i],kvals[j]*1e3)
        ELF_Lindhard[i][j] = Si.elf(ovals[i],kvals[j]*1e3,method="Lindhard")

fig, axs = plt.subplots(2, 4,figsize=(14, 7))
rc('xtick', labelsize=14)
rc('ytick', labelsize=14)
plt.subplots_adjust(top = 0.93, bottom=0.01, hspace=0.1, wspace=0.2)

# first row, plots as function of omega
#######################################
# omega and k grid
olist=np.linspace(0,100,200)
klist=[ 1.0, 2.0e3, 5.0e3 , 7.0e3 ]
   
# loop over the momenta
for i in range(len(klist)):
    k=klist[i]
    
    # calculate ELF as function of omega
    #ELF_gpaw=list(map(lambda om: Si.elf(om,k),olist))
    ELF_mermin=list(map(lambda om: Si.elf(om,k),olist))
    ELF_Lindhard=list(map(lambda om: Si.elf(om,k,method="Lindhard"),olist))

    # plot
    #axs[0][i].plot(olist,ELF_gpaw,
    #               label="GPAW",
    #              color="Indigo")
    axs[0][i].plot(olist,ELF_mermin,
                   label=r"Mermin",
                   color="blue")
    axs[0][i].plot(olist,ELF_Lindhard,
                   label=r"Lindhard",
                   color="red")
    axs[0][i].set_xlabel(r"$\omega$ (eV)",fontsize=16)
    axs[0][i].set_xlim([0,100])
    axs[0][i].tick_params(bottom=True,direction='in',which='both')
    
axs[0][0].set_ylim([0,6])
axs[0][1].set_ylim([0,4])
axs[0][2].set_ylim([0,0.45])
axs[0][3].set_ylim([0,0.2])
axs[0][0].legend(loc="lower right",framealpha=1)
axs[0][0].set_ylabel(r'$\mathrm{ELF}(\omega,k)$')#set_ylabel(r"$\mathrm{Im}\left[\frac{-1}{\epsilon(\omega,k)}\right]$",fontsize=16)

# add some decoration
for i in range(4):
    axs[0][i].text(0.05,0.9,"Si",fontsize=16,transform = axs[0][i].transAxes)
    axs[0][i].text(0.95,0.9,r"$k=$ "+str(np.round(klist[i]*1e-3,0))+" keV",transform = axs[0][i].transAxes,fontsize=16,horizontalalignment="right")
    

    
# second row, plots as function of k
#######################################    
    
# omega and k grid
olist=np.linspace(0,100,200)
klist=[ 1.0, 2.0e3, 5.0e3 , 7.0e3 ]
   
# loop over the momenta
for i in range(len(klist)):
    k=klist[i]
    
    # calculate ELF as function of omega
    #ELF_gpaw=list(map(lambda om: Ge.elf(om,k),olist))
    ELF_mermin=list(map(lambda om: Ge.elf(om,k),olist))
    ELF_Lindhard=list(map(lambda om: Ge.elf(om,k,method="Lindhard"),olist))

    # plot
    #axs[1][i].plot(olist,ELF_gpaw,
    #               label="GPAW",
    #              color="Indigo")
    axs[1][i].plot(olist,ELF_mermin,
                   label=r"Mermin",
                   color="blue")
    axs[1][i].plot(olist,ELF_Lindhard,
                   label=r"Lindhard",
                   color="red")
    axs[1][i].set_xlabel(r"$\omega$ (eV)",fontsize=16)
    axs[1][i].set_xlim([0,100])
    axs[1][i].tick_params(bottom=True,direction='in',which='both')
    plt.grid()
    
#axs[0].set_ylim([0,6])
#axs[1].set_ylim([0,4])
#axs[2].set_ylim([0,0.45])
axs[1][0].set_ylim([0,4])
#axs[3].set_ylim([0,0.2])
axs[1][0].legend(loc='lower right',framealpha=1)
axs[1][0].set_ylabel(r'$\mathrm{ELF}(\omega,k)$')#(r"$\mathrm{Im}\left[\frac{-1}{\epsilon(\omega,k)}\right]$",fontsize=16)
#axs[0].set_xscale("Log")
#axs[1][0].set_yscale("Log")

# add some decoration
for i in range(4):
    axs[1][i].text(0.05,0.9,"Ge",fontsize=16,transform = axs[1][i].transAxes)
    axs[1][i].text(0.95,0.9,r"$k=$ "+str(np.round(klist[i]*1e-3,0))+" keV",transform = axs[1][i].transAxes,fontsize=16,horizontalalignment="right")

fig.tight_layout()     
    
#Saves figure as a pdf-file to the 'figures' folder
fig.savefig('figures/ELF.pdf',bbox_inches='tight')


In [ ]:
#Figure 2 - Contour plot for velocity distribution

#Constants
m_chi = 1e6 #Arbitrary DM-mass
omega = np.linspace(0,3*m_chi*v_0**2)
k     = np.linspace(0,3*m_chi*v_0)

#Contour graph routine
np.seterr(divide='ignore', invalid='ignore')
[K,W] = np.meshgrid(k,omega)

X = np.zeros((len(K), len(K[0])))
Y = np.zeros((len(K), len(K[0])))
contour_func = np.zeros((len(K), len(K[0])))

for i in range(len(K)):
        for j in range(len(K[0])):
                contour_func[i][j] = W[i][j]/K[i][j]*p1(W[i][j]/K[i][j] + K[i][j]/(2*m_chi))
                X[i][j] =  K[i][j]/(v_0*m_chi)
                Y[i][j] = W[i][j]/(v_0**2*m_chi) 

plt.contourf(X,Y,contour_func,9)
plt.xlabel(r'$k/(m_\chi v_0)$')
plt.ylabel(r'$\omega/(m_\chi v_0^2)$')
cbar = plt.colorbar()
cbar.set_label(r'$\frac{\omega}{k}p_1\left( \frac{\omega}{k} + \frac{k}{wm_\chi} \right) \mathrm{\; [s/km]}$')
plt.grid(False)
plt.show()

#Saves figure as a pdf-file to the 'figures' folder
plt.savefig('figures/velocitydist.pdf',bbox_inches='tight')

In [ ]:
#Figure 3 - Gamma-ratio using our derived gamma-bar

#Solid angle
solid = 1/(2*np.pi**2)#1/(2*np.pi)**3

#I(w,k) for massless mediator
def I(w,k):
      return (w/k*p1(w/k + k/(2*m_chi)))/w

#Lower energy bounds [eV]
w_min_al   = .01
w_max_al   = 1
w_min_si   = 4.7
w_min_ge   = 3.6
w_min_sio2 = 16.8

#Gamma-opt
def gamma_opt(m_X):
      return 0.68/(4*math.pi)*m_X*v_0/c

#Integral of I(w,k)ELF(w,k)
def Si_integrand():
      f = lambda k, w: I(w,k)*Si.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_si,w_max,k_min,k_max)[0] * eVtoInvYr

def Ge_integrand():
      f = lambda k, w: I(w,k)*Ge.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_ge,w_max,k_min,k_max)[0] * eVtoInvYr

def Al_integrand():
      f = lambda k, w: I(w,k)*Al.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_al,w_max_al,k_min,k_max)[0] * eVtoInvYr

def SiO2_integrand():
      f = lambda k, w: I(w,k)*SiO2.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_sio2,w_max,k_min,k_max)[0] * eVtoInvYr

#Gamma-bar 
def Si_gamma_bar():
      return 2*Si_integrand()*solid*c

def Ge_gamma_bar():
      return 2*Ge_integrand()*solid*c

def Al_gamma_bar():
      return 2*Al_integrand()*solid*c

def SiO2_gamma_bar():
      return 2*SiO2_integrand()*solid*c

#DM-mass
mX = np.logspace(2,9,50) #0.01 - 1000 MeV

Si_ratio   = []
Al_ratio   = []
Ge_ratio   = []
SiO2_ratio = []

#If the code has already been executed, data from previous session can be re-loaded
Si_ratio   = np.load('OurData/Gamma-bar Si.npy')
Ge_ratio   = np.load('OurData/Gamma-bar Ge.npy')
Al_ratio   = np.load('OurData/Gamma-bar Al.npy')
SiO2_ratio = np.load('OurData/Gamma-bar SiO2.npy')
'''
#Routine for calculating gamma_opt/gamma-bar for all different mX
step = 0
for x in mX:
      step = step + 1
      clear_output(wait=True)
      print('Working... (',step,'/',len(mX),')',sep='')
      
      m_chi = x
      w_max = (v_max**2 * m_chi / 2)
      k_min = lambda w: (v_max*m_chi - np.sqrt(v_max**2 * m_chi**2 - 2 * m_chi * w))
      k_max = lambda w: (v_max*m_chi + np.sqrt(v_max**2 * m_chi**2 - 2 * m_chi * w))

      Si.update_params(mX = m_chi,mediator = 'massless')
      Al.update_params(mX = m_chi,mediator = 'massless')
      Ge.update_params(mX = m_chi,mediator = 'massless')
      SiO2.update_params(mX = m_chi,mediator = 'massless')

      Si_ratio.append(gamma_opt(m_chi)/Si_gamma_bar())
      Al_ratio.append(gamma_opt(m_chi)/Al_gamma_bar())
      Ge_ratio.append(gamma_opt(m_chi)/Ge_gamma_bar())
      SiO2_ratio.append(gamma_opt(m_chi)/SiO2_gamma_bar())

clear_output(wait=True)

#Saves data for future use
np.save('OurData/Gamma-bar Si.npy',Si_ratio)
np.save('OurData/Gamma-bar Ge.npy',Ge_ratio)
np.save('OurData/Gamma-bar Al.npy',Al_ratio)
np.save('OurData/Gamma-bar SiO2.npy',SiO2_ratio)
'''
mX = mX/1e6

#Graphing
plt.loglog(mX,Si_ratio,color = 'red',label = 'Si')
plt.loglog(mX,Al_ratio,color = 'green',label = 'Al')
plt.loglog(mX,Ge_ratio,color = 'blue',label ='Ge')
plt.loglog(mX,SiO2_ratio,color = 'orange',label ='SiO2')
plt.xlabel(r'$m_\mathrm{\chi} \mathrm{\; [MeV]}$')
plt.ylabel(r'$\bar{\Gamma}_{\mathrm{opt}}/\bar{\Gamma}$')
plt.legend(framealpha=1)
plt.grid(color='gainsboro')
plt.xlim([1e-1,1e3])
plt.ylim([.5e0,5e2])
plt.axhline(1,0,1e4,color='black')
plt.show

#Saves figure as a pdf-file to the 'figures' folder
plt.savefig('figures/Gamma-ratio using Gamma-bar.pdf',bbox_inches='tight')

In [ ]:
#Figure 4 - Gamma-ratio using target.R_electron

scalar = 916.667 #g_chi^2*g_e^2/e^2

#Gamma-opt
def gamma_opt(m_X):
    return 0.68/(4*math.pi)*m_X*v_0/c*scalar

#If the code has already been executed, the previous data can be reloaded instead
R_SI = np.load('OurData/R Si.npy')
R_Ge = np.load('OurData/R Ge.npy')
R_Al = np.load('OurData/R Al.npy')
R_SiO2 = np.load('OurData/R SiO2.npy')
mX = np.load('OurData/R mX.npy')

'''
#DM mass
mX = np.logspace(4,9,100) #0.01 - 1000 MeV

R_Si = []
R_Al = []
R_Ge = []
R_SiO2 = []

#Routine for calculating Gamma_opt/R for each mX
step = 0
for x in mX:
    step = step + 1
    clear_output(wait=True)
    print('Working... (',step,'/',len(mX),')',sep='')

    Si.update_params(mX = x,mediator = 'massless')
    Al.update_params(mX = x,mediator = 'massless')
    Ge.update_params(mX = x,mediator = 'massless')
    SiO2.update_params(mX = x,mediator = 'massless')

    R_Si.append(gamma_opt(x)/Si.R_electron())
    R_Al.append(gamma_opt(x)/Al.R_electron(threshold=0.01,withscreening=True))
    R_Ge.append(gamma_opt(x)/Ge.R_electron())
    R_SiO2.append(gamma_opt(x)/SiO2.R_electron())

mX = mX/1e6

#Saves data for future use
np.save('OurData/R Si.npy',R_Si)
np.save('OurData/R Ge.npy',R_Ge)
np.save('OurData/R Al.npy',R_Al)
np.save('OurData/R SiO2.npy',R_SiO2)
np.save('OurData/R mX.npy',mX)
'''
#Graphing routine
plt.loglog(mX,R_Si,color='red',label='Si')
plt.loglog(mX,R_Al,color='green',label='Al')
plt.loglog(mX,R_Ge,color='blue',label='Ge')
plt.loglog(mX,R_SiO2,color='orange',label='SiO2')
plt.axhline(1,0,1e4,color='black')
plt.xlabel(r'$m_\mathrm{\chi} \textrm{ [MeV]}$')
plt.ylabel(r'$\bar{\Gamma}_{\mathrm{opt}}/\bar{\Gamma}$')
plt.ylim([0.5,1e7])
plt.legend(framealpha=1)
plt.grid(color='gainsboro')
plt.show

#Saves the figure as a pdf-file to the 'figures' folder
plt.savefig('figures/Gamma-ratio using R.pdf',bbox_inches='tight')